# import section

In [95]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense ,Activation , Dropout
from tensorflow.keras.optimizers import SGD
import random

In [96]:
words = []
classes = []
documents = []
ignore_words = ['!' , '?']
data_file = open('intents.json',encoding="utf8").read()
intents = json.loads(data_file)

### intents file:
{

"intents": [
    
{"tag": "greeting",

"patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],

"responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],

"context_set": ""

}, ... ] }

## get data out of the intent to our list

In [97]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
    if intent['tag'] not in classes:
        classes.append(intent['tag'])
print(classes)

['greeting', 'goodbye', 'name', 'gpa', 'gpa2', 'credithours', 'credithours2', 'specialize', 'Math211', 'Math231', 'Math271', 'Cs201', 'Stat201', 'Cs202', 'Math272', 'Math232', 'Math213', 'Math212', 'Math241', 'Cs302', 'Cs304', 'Cs305', 'Cs307', 'Cs309', 'Math305', 'Math351', 'Math355', 'Math361', 'Cs316', 'Cs303', 'Cs306', 'Cs308', 'Math306', 'Math352', 'Math356', 'Math362', 'Cs317', 'Math481', 'Math482', 'Math442', 'Math441', 'Cs401', 'Cs403', 'Cs406', 'Cs426', 'Cs428', 'Cs490', 'Cs427', 'Cs408', 'Cs407', 'Cs402']


example :

documents = > 
[ (['Hi'], 'greeting') , (['How', 'are', 'you'], 'greeting') , (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting')]

words => ['Hi', 'How', 'are', 'you', 'Is']

classes = > ['greeting', 'goodbye', 'name', 'gpa', 'gpa2']

In [98]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if  w not in ignore_words]
# get red of dublications 
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(len(documents),' documents')
print(len(classes),' classes')
print(len(words),' unique lemmatized words')

509  documents
51  classes
205  unique lemmatized words


In [99]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

## create train and testing data

In [100]:
training = []
output_empty = [0] * len(classes)
## 
# make bag of word for each doc
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(w.lower()) for w in pattern_words if w not in ignore_words]
                   
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag , output_row])
    

In [101]:
# random the data
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


C:\Users\mahmoud\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [102]:
model = Sequential()
model.add(Dense(128,input_shape = (len(train_x[0]),), activation = 'relu' ))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu' ))
model.add(Dropout(0.5))
model.add(Dense( len(train_y[0]), activation = 'softmax' ))

sgd = SGD(learning_rate=0.01, decay=1e-6 , momentum=0.9,nesterov = True)
model.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])

# train the model and save 
hist = model.fit(np.array(train_x),np.array(train_y),epochs=200 , batch_size = 5 , verbose = 1)
model.save('chat_model.h5',hist)

Epoch 1/200
102/102 [==============================] - 1s 1ms/step - loss: 3.9650 - accuracy: 0.0079
Epoch 2/200
102/102 [==============================] - 0s 1ms/step - loss: 3.9038 - accuracy: 0.0295
Epoch 3/200
102/102 [==============================] - 0s 1ms/step - loss: 3.9039 - accuracy: 0.0255
Epoch 4/200
102/102 [==============================] - 0s 2ms/step - loss: 3.8631 - accuracy: 0.0354
Epoch 5/200
102/102 [==============================] - 0s 2ms/step - loss: 3.8723 - accuracy: 0.0334
Epoch 6/200
102/102 [==============================] - 0s 2ms/step - loss: 3.8107 - accuracy: 0.0432
Epoch 7/200
102/102 [==============================] - 0s 1ms/step - loss: 3.8098 - accuracy: 0.0472
Epoch 8/200
102/102 [==============================] - 0s 2ms/step - loss: 3.7291 - accuracy: 0.0707
Epoch 9/200
102/102 [==============================] - 0s 2ms/step - loss: 3.6588 - accuracy: 0.0707
Epoch 10/200
102/102 [==============================] - 0s 2ms/step - loss: 3.5754 - accura

Epoch 81/200
102/102 [==============================] - 0s 2ms/step - loss: 0.5333 - accuracy: 0.8389
Epoch 82/200
102/102 [==============================] - 0s 2ms/step - loss: 0.5114 - accuracy: 0.8409
Epoch 83/200
102/102 [==============================] - 0s 2ms/step - loss: 0.5733 - accuracy: 0.8232
Epoch 84/200
102/102 [==============================] - 0s 2ms/step - loss: 0.5268 - accuracy: 0.8468
Epoch 85/200
102/102 [==============================] - 0s 2ms/step - loss: 0.3846 - accuracy: 0.8802
Epoch 86/200
102/102 [==============================] - 0s 2ms/step - loss: 0.5329 - accuracy: 0.8448
Epoch 87/200
102/102 [==============================] - 0s 3ms/step - loss: 0.5769 - accuracy: 0.8507
Epoch 88/200
102/102 [==============================] - 0s 2ms/step - loss: 0.5043 - accuracy: 0.8625
Epoch 89/200
102/102 [==============================] - 0s 2ms/step - loss: 0.4396 - accuracy: 0.8585
Epoch 90/200
102/102 [==============================] - 0s 2ms/step - loss: 0.4200

102/102 [==============================] - 0s 1ms/step - loss: 0.3437 - accuracy: 0.9077
Epoch 161/200
102/102 [==============================] - 0s 1ms/step - loss: 0.3800 - accuracy: 0.8703
Epoch 162/200
102/102 [==============================] - 0s 1ms/step - loss: 0.6036 - accuracy: 0.8350
Epoch 163/200
102/102 [==============================] - 0s 1ms/step - loss: 0.4282 - accuracy: 0.8782
Epoch 164/200
102/102 [==============================] - 0s 1ms/step - loss: 0.4815 - accuracy: 0.8644
Epoch 165/200
102/102 [==============================] - 0s 1ms/step - loss: 0.4670 - accuracy: 0.8703
Epoch 166/200
102/102 [==============================] - 0s 1ms/step - loss: 0.3693 - accuracy: 0.8978
Epoch 167/200
102/102 [==============================] - 0s 1ms/step - loss: 0.4028 - accuracy: 0.8880
Epoch 168/200
102/102 [==============================] - 0s 2ms/step - loss: 0.4301 - accuracy: 0.8939
Epoch 169/200
102/102 [==============================] - 0s 2ms/step - loss: 0.5215 - a

# make predections

In [103]:
import nltk 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chat_model.h5')
import json 
import random

intents = json.loads(open('intents.json',encoding="utf8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
print(classes)

['Cs201', 'Cs202', 'Cs302', 'Cs303', 'Cs304', 'Cs305', 'Cs306', 'Cs307', 'Cs308', 'Cs309', 'Cs316', 'Cs317', 'Cs401', 'Cs402', 'Cs403', 'Cs406', 'Cs407', 'Cs408', 'Cs426', 'Cs427', 'Cs428', 'Cs490', 'Math211', 'Math212', 'Math213', 'Math231', 'Math232', 'Math241', 'Math271', 'Math272', 'Math305', 'Math306', 'Math351', 'Math352', 'Math355', 'Math356', 'Math361', 'Math362', 'Math441', 'Math442', 'Math481', 'Math482', 'Stat201', 'credithours', 'credithours2', 'goodbye', 'gpa', 'gpa2', 'greeting', 'name', 'specialize']


In [104]:
def clean_up_sentence(sent):
    sentence_words = nltk.word_tokenize(sent)
    sentence_word = [lemmatizer.lemmatize(w.lower()) for w in sentence_words]
    return sentence_words

In [105]:
def bow(sentence , words , show_details = True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i , w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print('found in beg: %s' % w)
    return (np.array(bag))

In [106]:
def predict_class(sentence , model):
    p = bow(sentence , words, show_details = False)
    res = model.predict(np.array([p]))[0]
    print(res)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD ]
    print(results)
    results.sort(key=lambda x: x[1] , reverse = True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

In [108]:
predict_class('how specialize',model)

[7.9124968e-04 1.1117439e-05 9.6928825e-06 4.4764884e-04 1.1272946e-06
 8.5448199e-05 5.9864567e-05 2.1933780e-05 1.5108005e-04 7.8775112e-05
 5.1605381e-04 2.9535190e-06 2.5100110e-04 1.9247152e-03 4.8894250e-05
 9.1372326e-04 4.6552108e-05 6.1680941e-04 1.0105707e-06 7.4830285e-07
 2.1515606e-04 2.9599803e-05 4.4446480e-03 4.6035131e-03 1.6469385e-05
 8.4125371e-05 5.0151505e-07 1.2626310e-05 1.2590676e-04 1.8886817e-03
 2.2118927e-03 8.8341776e-03 2.3919147e-04 6.4549400e-05 1.2608351e-06
 2.5910547e-04 1.1188297e-05 1.9224141e-04 6.9696862e-06 1.7439619e-05
 7.7554034e-07 4.0680080e-04 1.8782941e-05 9.7200699e-04 7.6480895e-01
 1.0209508e-01 3.9212587e-03 4.4521652e-03 9.0679377e-02 2.9162255e-03
 4.8890989e-04]
[[44, 0.76480895]]


[{'intent': 'credithours2', 'probability': '0.76480895'}]

In [94]:
def getResponse(ints,intents_json):
    result =''
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag']==tag):
            result = random.choice(i['responses'])
            break
    return result


In [66]:
def chatbot_response(msg):
    ints = predict_class(msg,model)
    print(ints)
    res = getResponse(ints,intents)
    
    return res

In [84]:
chatbot_response('hi')

[{'intent': 'greeting', 'probability': '1.0'}]


'Hello!'